<a href="https://colab.research.google.com/github/dhrubo020/NLP_01/blob/master/banglish_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import functools

import numpy as np
import tensorflow as tf
import gensim, re
import pandas as pd

In [0]:
train_data = ['banglish.csv']

In [22]:
"""
import csv

filename = 'banglish.csv'
raw_data = open(filename, mode='r')
reader = csv.reader(raw_data, delimiter=',', quoting=csv.QUOTE_NONE)
x = list(reader)
train_data = np.array(x).astype('str')
print(train_data[1])
"""

from google.colab import files 
#Use to import io, which opens the file from the Colab drive
import io

# This will open a widget when run that will enable you to browse the files on your local storage drive.
uploaded = files.upload()

# Now use the below code to read the csv in pandas' dataframe
train_data = pd.read_csv(io.StringIO(uploaded['banglish.csv'].decode('ISO-8859-1')))


Saving banglish.csv to banglish (8).csv


In [0]:
train_data = np.array(train_data)

In [0]:
text = []
label = []
i=0
for x in train_data:
    text.append(x[0])
    if(x[1]==-1):
      label.append(2)
    else:
      label.append(x[1])

In [0]:
#text = re.sub(r"[,.\"!@#$%^&*(){}?/;`~:<>+=-]", '', text)
text=np.array(text)
#print(label[5090:])

In [111]:
# text cleaning.......
import re
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

def clean_text(df):
    all_reviews = []
    #lines = df["Review_text"].values.tolist()
    for text in df:
        text = text.lower()
        
        pattern = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
        text = pattern.sub('', text)
        
        emoji = re.compile("["
                           u"\U0001F600-\U0001FFFF"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
        text = emoji.sub(r'', text)
        
        text = re.sub(r"i'm", "i am", text)
        text = re.sub(r"he's", "he is", text)
        text = re.sub(r"she's", "she is", text)
        text = re.sub(r"that's", "that is", text)        
        text = re.sub(r"what's", "what is", text)
        text = re.sub(r"where's", "where is", text) 
        text = re.sub(r"\'ll", " will", text)  
        text = re.sub(r"\'ve", " have", text)  
        text = re.sub(r"\'re", " are", text)
        text = re.sub(r"\'d", " would", text)
        text = re.sub(r"\'ve", " have", text)
        text = re.sub(r"won't", "will not", text)
        text = re.sub(r"don't", "do not", text)
        text = re.sub(r"did't", "did not", text)
        text = re.sub(r"can't", "can not", text)
        text = re.sub(r"it's", "it is", text)
        text = re.sub(r"couldn't", "could not", text)
        text = re.sub(r"have't", "have not", text)
        
        text = re.sub(r"[,.\"!@#$%^&*(){}?/;`~:<>+=-]", "", text)
        
        tokens = word_tokenize(text)
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokens]
        words = [word for word in stripped if word.isalpha()]
#         stop_words = set(stopwords.words("english"))
#         stop_words.discard("not")
#         words = [w for w in words if not w in stop_words]
        words = ' '.join(words)
        all_reviews.append(words)
    return all_reviews

text = clean_text(text)
text[0:20]

['allah uni to amr hall r apu',
 'amader ektai dabi amader desher moto niriho islam prio manush jeno hoyranir shikar na hoy mr erdogan e bepare shocheton asen bole mone kori',
 'aj ki kono mach asay',
 'bjibike fair er wodar dey na ken',
 'amra tho bograi taki tahole dakay oto taka koros kore jabo ki kore',
 'muslim hisabe amader uchit oder asroy deya dut amra ki asolei muslim rasthe baskokri',
 'train e cole aste bolenn',
 'hahaha bolar kichu nai sob gayni vai der comment ai khane',
 'movie na jene r kotha na bola e uchitalia er hero onno actor',
 'jr jr icha r tara bujha e korcha ti nah ata neya amra matha gamanor ki dorkr abr adr movie chara amra toh cholta pari nah tila kn ato base korbo',
 'filmeto sobkicui hoi r atate age ki jai ase',
 'hi sob ke a deya amra ke korbo',
 'amer je onk kisu churi holo ta ke uddar karbe',
 'hahhaha ki je bolen dada',
 'dukhu kostu niye amader jibon ma go apnk salamduia kori allah apnr kosto dur kore den',
 'vi tomar kotha suna mona holo tumi tablik j

In [0]:
text = [re.sub(r'[,.\"!@#$%^&*(){}?/;`~:<>+=-]+', '', sentence) for sentence in text]

In [0]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import gensim, re
import pandas as pd


features = 1000
tokenizer = Tokenizer(num_words = features)

# আমাদের টোকেনাইজারকে পুরো টেক্সটে ফিট করতে হবে 
tokenizer.fit_on_texts(text)

# আমাদের টোকেনাইজার চেনে সেরকম শব্দ নিয়ে আসতে হবে 
word_index = tokenizer.word_index

# ম্যাট্রিক্স এর মধ্যে টোকেনকে ফেলি 
X = tokenizer.texts_to_sequences(text)
X = pad_sequences(X,maxlen=15)

# লেবেল তৈরি করি 
y = np.asarray(pd.get_dummies(label))


In [137]:
print(word_index)

{'na': 1, 'ki': 2, 'ta': 3, 'valo': 4, 'er': 5, 'r': 6, 'e': 7, 'kore': 8, 'to': 9, 'k': 10, 'a': 11, 'ar': 12, 'kotha': 13, 'jonno': 14, 'hobe': 15, 'ami': 16, 'kono': 17, 'vai': 18, 'amader': 19, 'ke': 20, 'hoy': 21, 'korte': 22, 'o': 23, 'ei': 24, 'onek': 25, 'sob': 26, 'amra': 27, 'kora': 28, 'bangladesh': 29, 'akta': 30, 'der': 31, 'apnar': 32, 'moto': 33, 'tai': 34, 'jodi': 35, 'theke': 36, 'ra': 37, 'mone': 38, 'tader': 39, 'tar': 40, 'khub': 41, 'nai': 42, 'amar': 43, 'din': 44, 'allah': 45, 'pare': 46, 'bole': 47, 'je': 48, 'ai': 49, 'hoi': 50, 'muslim': 51, 'ase': 52, 'ato': 53, 'sathe': 54, 'tahole': 55, 'boro': 56, 'apni': 57, 'hole': 58, 'kichu': 59, 'te': 60, 'shakib': 61, 'hoye': 62, 'tara': 63, 'korar': 64, 'hok': 65, 'taka': 66, 'kori': 67, 'tumi': 68, 'aro': 69, 'ti': 70, 'desh': 71, 'age': 72, 'oder': 73, 'thik': 74, 'ja': 75, 'ekta': 76, 'koto': 77, 'tor': 78, 'chai': 79, 'niye': 80, 'keno': 81, 'uchit': 82, 'por': 83, 'j': 84, 'kharap': 85, 'sakib': 86, 'holo': 87,

In [138]:
pd.get_dummies(label)

,0.0,1.0,2.0
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
...,...,...,...
5997,0,0,1
5998,0,0,1
5999,0,0,1
6000,0,0,1


In [0]:
#pd.get_dummies(label[:2000])
#pd.get_dummies(label[2001:4000])
#pd.get_dummies(label[4000:])


In [140]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

X_train.shape

(4801, 15)

In [0]:
lines_from_file = []

for line in text:
    lines_from_file.append(line)
      
tokenized_lines = []
for single_line in lines_from_file:
    tokenized_lines.append(single_line.split())

In [143]:
print(tokenized_lines[:10])

[['allah', 'uni', 'to', 'amr', 'hall', 'r', 'apu'], ['amader', 'ektai', 'dabi', 'amader', 'desher', 'moto', 'niriho', 'islam', 'prio', 'manush', 'jeno', 'hoyranir', 'shikar', 'na', 'hoy', 'mr', 'erdogan', 'e', 'bepare', 'shocheton', 'asen', 'bole', 'mone', 'kori'], ['aj', 'ki', 'kono', 'mach', 'asay'], ['bjibike', 'fair', 'er', 'wodar', 'dey', 'na', 'ken'], ['amra', 'tho', 'bograi', 'taki', 'tahole', 'dakay', 'oto', 'taka', 'koros', 'kore', 'jabo', 'ki', 'kore'], ['muslim', 'hisabe', 'amader', 'uchit', 'oder', 'asroy', 'deya', 'dut', 'amra', 'ki', 'asolei', 'muslim', 'rasthe', 'baskokri'], ['train', 'e', 'cole', 'aste', 'bolenn'], ['hahaha', 'bolar', 'kichu', 'nai', 'sob', 'gayni', 'vai', 'der', 'comment', 'ai', 'khane'], ['movie', 'na', 'jene', 'r', 'kotha', 'na', 'bola', 'e', 'uchitalia', 'er', 'hero', 'onno', 'actor'], ['jr', 'jr', 'icha', 'r', 'tara', 'bujha', 'e', 'korcha', 'ti', 'nah', 'ata', 'neya', 'amra', 'matha', 'gamanor', 'ki', 'dorkr', 'abr', 'adr', 'movie', 'chara', 'amra

In [0]:
word_model = gensim.models.Word2Vec(tokenized_lines, size=500, min_count=1, iter=20)
#word_model = gensim.models.Word2Vec(text, size=300, min_count=1, iter=10)

In [151]:
word_model.wv.most_similar(positive=['amr'])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('dami', 0.9984883069992065),
 ('aponar', 0.9980617761611938),
 ('hoyece', 0.9978435039520264),
 ('potrika', 0.9977854490280151),
 ('suna', 0.9977520704269409),
 ('kub', 0.9971354007720947),
 ('jibonta', 0.9969334006309509),
 ('story', 0.9969220161437988),
 ('kkr', 0.9965271949768066),
 ('koster', 0.9963749051094055)]

In [0]:
# নতুন ম্যাট্রিক্সে সেভ করি ভেক্টরগুলোকে 
embedding_matrix = np.zeros((len(word_model.wv.vocab) + 1, 500))
for i, vec in enumerate(word_model.wv.vectors):
    embedding_matrix[i] = vec

In [153]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding

# লেয়ারকে ইনিশিয়ালাইজ করি, অবশ্যই সিকোয়েন্সিয়াল মডেল  
model = Sequential()
# শব্দ ভেক্টরকে এমবেড করি 
model.add(Embedding(len(word_model.wv.vocab)+1, 500, input_length=X.shape[1], weights=[embedding_matrix], trainable=False))
# ডেটার মধ্যে কো-রিলেশন বোঝার চেষ্টা করতে দেই মডেলকে 
model.add(LSTM(100,return_sequences=False))
model.add(Dense(y.shape[1],activation="softmax"))
# মডেলের কাঠামো দেখি 
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 15, 500)           6774000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               240400    
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 303       
Total params: 7,014,703
Trainable params: 240,703
Non-trainable params: 6,774,000
_________________________________________________________________


In [0]:
# মডেলকে কম্পাইল করি 

model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=['acc'])

In [159]:
# মডেলকে ট্রেইন করি ব্যাচ দিয়ে, দেখানোর জন্য

batch = 32
epochs = 20
model.fit(X_train, y_train, batch, epochs)

Epoch 1/20
151/151 [==============================] - 4s 30ms/step - loss: 0.4999 - acc: 0.8017
Epoch 2/20
151/151 [==============================] - 5s 30ms/step - loss: 0.4640 - acc: 0.8225
Epoch 3/20
151/151 [==============================] - 5s 32ms/step - loss: 0.4540 - acc: 0.8317
Epoch 4/20
151/151 [==============================] - 5s 30ms/step - loss: 0.4375 - acc: 0.8371
Epoch 5/20
151/151 [==============================] - 5s 30ms/step - loss: 0.5327 - acc: 0.7817
Epoch 6/20
151/151 [==============================] - 4s 30ms/step - loss: 0.4071 - acc: 0.8515
Epoch 7/20
151/151 [==============================] - 4s 29ms/step - loss: 0.3715 - acc: 0.8715
Epoch 8/20
151/151 [==============================] - 4s 30ms/step - loss: 0.3896 - acc: 0.8627
Epoch 9/20
151/151 [==============================] - 5s 30ms/step - loss: 0.4041 - acc: 0.8461
Epoch 10/20
151/151 [==============================] - 4s 30ms/step - loss: 0.3306 - acc: 0.8911
Epoch 11/20
151/151 [==================

In [170]:
model.evaluate(X_test, y_test)

38/38 [==============================] - 0s 12ms/step - loss: 2.0423 - acc: 0.4221


[2.042349100112915, 0.422148197889328]

In [0]:
sents = ['valo lage na','Allah Uni to amr hall r apu', 'amr khub kharap lagce', 'onek sundor', 'ami ar ki bolbo somay asla bolbo', 'dhur sala', 'khub valo hoiche']
# sent_n = [[word_index[w]+3 for w in s.split()] for s in sents]
sent_n = tokenizer.texts_to_sequences(sents)
X_new = pad_sequences(sent_n,maxlen=15)
prediction = model.predict(X_new)

In [178]:
# '১' মানে পজিটিভ, '০' মানে নেগেটিভ, কারণ pd.get_dummy() দেখিয়েছে কলাম '০' মানে নেগেটিভ আর কলাম '১' মানে পজিটিভ  

np.argmax(prediction, axis=1)

array([2, 0, 2, 1, 0, 1, 1])